# Building a Deep Convolutional Neural Network

## Understanding convolution

A *convolution* is a multiplication between two matrices, with one matrix being big and the other smaller. Consider the following example. Matrix A is as follows:

<img src="../images/matrixA.png" width="240px" alt="" align="left">
<br clear="all" />

Matrix B is as follows:

<img src="../images/matrixB.png" width="130px" alt="" align="left">
<br clear="all" />

While performing convolution, think of it as sliding the smaller matrix over the bigger matrix: we can potentially come up with nine such multiplications as the smaller matrix is slid over the entire area of the bigger matrix. Note that it is *not* matrix multiplication:

* `{1,2,5,6}` of the bigger matrix is multiplied with `{1,2,3,4}` of the smaller matrix: 
`1×1 + 2×2 + 5×3 + 6×4 = 44`
* `{2,3,6,7}` of the bigger matrix is multiplied with `{1,2,3,4}` of the smaller matrix: 
`2×1 + 3×2 + 6×3 + 7×4 = 54`
* `{3,4,7,8}` of the bigger matrix is multiplied with `{1,2,3,4}` of the smaller matrix: 
`3×1 + 4×2 + 7×3 + 8×4 = 64`
* `{5,6,9,10}` of the bigger matrix is multiplied with `{1,2,3,4}` of the smaller matrix: 
`5×1 + 6×2 + 9×3 + 10×4 = 84`
* `{6,7,10,11}` of the bigger matrix is multiplied with `{1,2,3,4}` of the smaller matrix: 
`6×1 + 7×2 + 10×3 + 11×4 = 94`
* `{7,8,11,12}` of the bigger matrix is multiplied with `{1,2,3,4}` of the smaller matrix: 
`7×1 + 8×2 + 11×3 + 12×4 = 104  `
* `{9,10,13,14}` of the bigger matrix is multiplied with `{1,2,3,4}` of the smaller matrix: 
`9×1 + 10×2 + 13×3 + 14×4 = 124 `
* `{10,11,14,15}` of the bigger matrix is multiplied with `{1,2,3,4}` of the smaller matrix: 
`10×1 + 11×2 + 14×3 + 15×4 = 134 `
* `{11,12,15,16}` of the bigger matrix is multiplied with `{1,2,3,4}` of the smaller matrix: 
`11×1 + 12×2 + 15×3 + 16×4 = 144`

The result of the preceding steps would be a matrix, as follows:

<img src="../images/matrixC.png" width="180px" alt="" align="left">
<br clear="all" />

Conventionally, the smaller matrix is called a *filter* or *kernel*, and the smaller matrix values can be considered as the constituent weights. 

### From Convolution to Activation

In a traditional NN, a hidden layer not only multiplies the input values by the weights, but also applies a non-linearity to the data, i.e. it passes the values through an *activation function*. A similar activity happens in a typical CNN, where the convolution is passed through an activation function. 

For the preceding output, note that the output remains the same when passed through a `ReLU` activation function, as all the numbers are positive. 

### From Convolution Activation to Pooling

Let’s say the output of the convolution step is as follows:

<img src="../images/matrixD.png" width="240px" alt="" align="left">
<br clear="all" />

*Max pooling* divides the matrix into non-overlapping blocks of size `2×2`:

<img src="../images/matrixE.png" width="240px" alt="" align="left">
<br clear="all" />

From each block, only the element that has the highest value is chosen. So, the output of the *max pooling* operation on the preceding matrix would be:

<img src="../images/matrixF.png" width="120px" alt="" align="left">
<br clear="all" />

### How Do Convolution and Pooling Help?

One of the drawbacks of traditional NN in the MNIST example is that each pixel is associated with a distinct weight. Thus, if an adjacent pixel other than the original pixel became highlighted, the output would not be very accurate (the example of scenario 1).

This scenario is now addressed, as the pixels share weights that are constituted within each filter. All the pixels are multiplied by all the weights that constitute a filter, and in the pooling layer only the values that are activated the highest are chosen. This way, regardless of whether the highlighted pixel is located at the center or slightly away from the center, the output would be more often the expected value.

## Building a CNN from scratch using Python 

Let's implement CNN with a *toy* example where the input and expected output data is defined from scratch using NumPy.

1. Create the dataset consisting of two inputs that are `4 × 4` in size and two outputs:

In [1]:
import numpy as np
X_train=np.array([[[1,2,3,4],[2,3,4,5],[5,6,7,8],[1,3,4,5]],[[-1,2,3,-4],[2,-3,4,5],[-5,6,-7,8],[-1,-3,-4,-5]]])
y_train=np.array([0,1])

In [2]:
X_train.shape

(2, 4, 4)

In [3]:
y_train.shape

(2,)

2. Scale the input dataset:

In [4]:
X_train = X_train / 8

3. Reshape the input dataset so that each input image is represented in the format of `number of samples x height x width x number of channels`:

In [5]:
X_train = X_train.reshape(X_train.shape[0],X_train.shape[1],X_train.shape[2],1).astype('float32')

In [6]:
X_train.shape

(2, 4, 4, 1)

4. Instantiate the model after importing the relevant methods:

In [7]:
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from keras.models import Sequential
model = Sequential()

5. Perform the *convolution* operation:

In [8]:
model.add(Conv2D(1, (3,3), input_shape=(4,4,1), activation='relu', name='Convolution_layer'))

In the preceding step, we perform a `2D` convolution on input data where we have `1` filter of size `3 x 3`. Additionally, given that this is the first layer, we specify the input shape. Finally, we perform `ReLU` activation on top of the output of the convolution.

The output of the convolution operation in this scenario is `2 x 2 x 1` in shape, as the matrix multiplication of weights with input would yield a `2 x 2` matrix.

6. Add a layer that performs a *max pooling* operation:

In [9]:
model.add(MaxPooling2D(pool_size=(2,2), name='Pooling_layer'))

In the preceding step, we perform *max pooling* on top of the output obtained from the previous layer, where the pool size is `2 x 2`. This means that the maximum value in a subset of size `2 x 2` from the image is calculated. 

7. Flatten the output from the pooling layer: 

In [10]:
model.add(Flatten(name='Flatten_layer'))

8. Feed the last output 1D vector into a densely connected classifier network like those we’re already familiar with: a stack of Dense layers. Here we directly connect the output of the flatten layer to the output layer using the `sigmoid` activation:

In [11]:
model.add(Dense(1, activation='sigmoid', name='Output_layer'))

Let’s display the architecture of the network:

In [12]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Convolution_layer (Conv2D)   (None, 2, 2, 1)           10        
_________________________________________________________________
Pooling_layer (MaxPooling2D) (None, 1, 1, 1)           0         
_________________________________________________________________
Flatten_layer (Flatten)      (None, 1)                 0         
_________________________________________________________________
Output_layer (Dense)         (None, 1)                 2         
Total params: 12
Trainable params: 12
Non-trainable params: 0
_________________________________________________________________


Note that there are `10` parameters in the *convolution layer* as the one `3 x 3` filter would have `9` weights and `1` bias term. The *pooling layer* and *flatten layer* do not have any parameters as they do not perform any operation where weights need to be modified.

The *output layer* has `2` parameters since the flatten layer has one output which is connected to the output layer that has `1` value; hence we will have one weight and one bias term connecting the flatten layer and output layer. 

9. Compile and fit the model:

In [13]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train, y_train, epochs = 500)

Epoch 1/500
1/1 [==============================] - 0s 456ms/step - loss: 0.6533 - accuracy: 0.5000
Epoch 2/500
1/1 [==============================] - 0s 4ms/step - loss: 0.6527 - accuracy: 0.5000
Epoch 3/500
1/1 [==============================] - 0s 4ms/step - loss: 0.6520 - accuracy: 0.5000
Epoch 4/500
1/1 [==============================] - 0s 5ms/step - loss: 0.6514 - accuracy: 0.5000
Epoch 5/500
1/1 [==============================] - 0s 9ms/step - loss: 0.6508 - accuracy: 0.5000
Epoch 6/500
1/1 [==============================] - 0s 4ms/step - loss: 0.6502 - accuracy: 0.5000
Epoch 7/500
1/1 [==============================] - 0s 8ms/step - loss: 0.6495 - accuracy: 0.5000
Epoch 8/500
1/1 [==============================] - 0s 3ms/step - loss: 0.6489 - accuracy: 0.5000
Epoch 9/500
1/1 [==============================] - 0s 3ms/step - loss: 0.6483 - accuracy: 0.5000
Epoch 10/500
1/1 [==============================] - 0s 4ms/step - loss: 0.6476 - accuracy: 0.5000
Epoch 11/500
1/1 [=========

1/1 [==============================] - 0s 8ms/step - loss: 0.5415 - accuracy: 1.0000
Epoch 167/500
1/1 [==============================] - 0s 4ms/step - loss: 0.5408 - accuracy: 1.0000
Epoch 168/500
1/1 [==============================] - 0s 2ms/step - loss: 0.5401 - accuracy: 1.0000
Epoch 169/500
1/1 [==============================] - 0s 9ms/step - loss: 0.5394 - accuracy: 1.0000
Epoch 170/500
1/1 [==============================] - 0s 3ms/step - loss: 0.5386 - accuracy: 1.0000
Epoch 171/500
1/1 [==============================] - 0s 2ms/step - loss: 0.5379 - accuracy: 1.0000
Epoch 172/500
1/1 [==============================] - 0s 2ms/step - loss: 0.5372 - accuracy: 1.0000
Epoch 173/500
1/1 [==============================] - 0s 3ms/step - loss: 0.5364 - accuracy: 1.0000
Epoch 174/500
1/1 [==============================] - 0s 2ms/step - loss: 0.5357 - accuracy: 1.0000
Epoch 175/500
1/1 [==============================] - 0s 4ms/step - loss: 0.5350 - accuracy: 1.0000
Epoch 176/500
1/1 [=====

1/1 [==============================] - 0s 2ms/step - loss: 0.4192 - accuracy: 1.0000
Epoch 331/500
1/1 [==============================] - 0s 8ms/step - loss: 0.4185 - accuracy: 1.0000
Epoch 332/500
1/1 [==============================] - 0s 3ms/step - loss: 0.4178 - accuracy: 1.0000
Epoch 333/500
1/1 [==============================] - 0s 3ms/step - loss: 0.4171 - accuracy: 1.0000
Epoch 334/500
1/1 [==============================] - 0s 8ms/step - loss: 0.4163 - accuracy: 1.0000
Epoch 335/500
1/1 [==============================] - 0s 3ms/step - loss: 0.4155 - accuracy: 1.0000
Epoch 336/500
1/1 [==============================] - 0s 6ms/step - loss: 0.4149 - accuracy: 1.0000
Epoch 337/500
1/1 [==============================] - 0s 3ms/step - loss: 0.4142 - accuracy: 1.0000
Epoch 338/500
1/1 [==============================] - 0s 3ms/step - loss: 0.4135 - accuracy: 1.0000
Epoch 339/500
1/1 [==============================] - 0s 4ms/step - loss: 0.4127 - accuracy: 1.0000
Epoch 340/500
1/1 [=====

1/1 [==============================] - 0s 7ms/step - loss: 0.3153 - accuracy: 1.0000
Epoch 494/500
1/1 [==============================] - 0s 4ms/step - loss: 0.3147 - accuracy: 1.0000
Epoch 495/500
1/1 [==============================] - 0s 10ms/step - loss: 0.3141 - accuracy: 1.0000
Epoch 496/500
1/1 [==============================] - 0s 3ms/step - loss: 0.3136 - accuracy: 1.0000
Epoch 497/500
1/1 [==============================] - 0s 4ms/step - loss: 0.3130 - accuracy: 1.0000
Epoch 498/500
1/1 [==============================] - 0s 12ms/step - loss: 0.3125 - accuracy: 1.0000
Epoch 499/500
1/1 [==============================] - 0s 7ms/step - loss: 0.3120 - accuracy: 1.0000
Epoch 500/500
1/1 [==============================] - 0s 2ms/step - loss: 0.3114 - accuracy: 1.0000


In the preceding code, we are specifying the loss as `binary cross-entropy` because the outcome is either a `1` or a `0`.

## Validating the CNN output

Let's validate the output we obtain from the model by implementing the feedforward portion of the CNN.

1. Extract the order in which weights and biases are presented: 

In [16]:
model.weights

[<tf.Variable 'Convolution_layer/kernel:0' shape=(3, 3, 1, 1) dtype=float32, numpy=
 array([[[[ 0.39993224]],
 
         [[-0.13413629]],
 
         [[ 0.29055312]]],
 
 
        [[[ 0.63828164]],
 
         [[ 0.8012859 ]],
 
         [[ 0.6382591 ]]],
 
 
        [[[-0.22340134]],
 
         [[ 0.68988985]],
 
         [[ 0.6218003 ]]]], dtype=float32)>,
 <tf.Variable 'Convolution_layer/bias:0' shape=(1,) dtype=float32, numpy=array([-0.31660166], dtype=float32)>,
 <tf.Variable 'Output_layer/kernel:0' shape=(1, 1) dtype=float32, numpy=array([[-0.8952714]], dtype=float32)>,
 <tf.Variable 'Output_layer/bias:0' shape=(1,) dtype=float32, numpy=array([0.63398004], dtype=float32)>]

Note that the weights of the *convolution layer* are presented first, then the bias, and finally the weight and bias in the *output layer*.

Taking into account that the image is `4 x 4 x 1` in shape, i.e. `image_height x image_width x image_channels`, we note that the shape of weights in the *convolution layer* is `(3, 3, 1, 1)` as the filter is `3 x 3 x 1` in shape and the final `1` (the fourth value in shape) is for the number of filters that are specified in the convolution layer.



2. Extract the weight values at various layers:

In [19]:
model.get_weights()

[array([[[[ 0.39993224]],
 
         [[-0.13413629]],
 
         [[ 0.29055312]]],
 
 
        [[[ 0.63828164]],
 
         [[ 0.8012859 ]],
 
         [[ 0.6382591 ]]],
 
 
        [[[-0.22340134]],
 
         [[ 0.68988985]],
 
         [[ 0.6218003 ]]]], dtype=float32),
 array([-0.31660166], dtype=float32),
 array([[-0.8952714]], dtype=float32),
 array([0.63398004], dtype=float32)]

In [20]:
model.get_weights()[0].shape

(3, 3, 1, 1)

3. Extract the output of the first input using the built model. We use the `predict` method to calculate the expected output:

In [32]:
X_train[0].shape

(4, 4, 1)

In [33]:
model.predict(X_train[0].reshape(1,4,4,1))

array([[0.17816815]], dtype=float32)

Now that we know the output is `0.17816815`, let’s validate our intuition of CNN so far by matching the preceding prediction through **feedforward propagation !!!**

4. Perform the convolution of the filter with the input image. Note that the input image is `4x4` in shape while the filter is `3x3` in shape. We will be performing the matrix multiplication in the *convolution layer*:

In [51]:
sumprod = []
imgL = X_train[0].shape[0] # image length
filL = model.get_weights()[0].shape[0] # filter length
model_weig = model.get_weights()[0].reshape(3,3) # filter weights
model_bias = model.get_weights()[1] # filter bias

for i in range(imgL-filL+1):
  for j in range(imgL-filL+1):
    img_subset = np.array(X_train[0,i:(i+3),j:(j+3),0])
    val = np.sum(img_subset*model_weig) + model_bias
    sumprod.append(val)
    
sumprod    

[array([1.5098622], dtype=float32),
 array([1.9751703], dtype=float32),
 array([1.9784077], dtype=float32),
 array([2.4157906], dtype=float32)]

5. Reshape `sumprod` and perform activation on top of the convolution's output: 

In [52]:
sumprod = np.array(sumprod).reshape(2,2,1) 
sumprod = np.where(sumprod>0,sumprod,0)

6. Pass the convolution's output to the *pooling layer*:

In [53]:
pooling_layer_output = np.max(sumprod)

7. Connect the pooling's output to the *output layer*:

In [54]:
intermediate_output_value = pooling_layer_output*model.get_weights()[2]+model.get_weights()[3]

8. Calculate the `sigmoid` output:

In [55]:
1/(1+np.exp(-intermediate_output_value))

array([[0.17816815]], dtype=float32)

This output is the same as the one that we obtained using the `model.predict` method, thus validating our understanding of how a CNN works !!!